Interesting link <a href="https://yanirseroussi.com/2015/07/06/learning-about-deep-learning-through-album-cover-classification/">Album Cover CNN </a> <a href="https://github.com/yanirs/bandcamp-deep-learning/">GIT</a>
<br />
Caffe example: <a href="https://github.com/yanirs/bandcamp-deep-learning/blob/master/notebooks/02-caffe-games.ipynb">Yanirs caffe2 pretrained model</a>
<br />
<a href="https://github.com/caffe2/models">Caffe2 models</a>
<br />
<a href="https://github.com/caffe2/tutorials/blob/master/Loading_Pretrained_Models.ipynb">Loading pre-trained models in Caffe2</a>

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os

from caffe2.python import core, model_helper, net_drawer, workspace, visualize, brew

In [3]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

1.13.1


# Obtaining the Dataset

In [4]:
import requests 
import csv
from multiprocessing.pool import ThreadPool


def _download_image(args):
    session, url, local_path = args[0], args[1], args[2]
    try:
        response = session.get(url)
    except requests.RequestException:
        return False
    if response.status_code != requests.codes.ok:
        return False

    with open(local_path, 'wb') as out:
        out.write(response.content)
    return True

def download_dataset_images(out_dir, dataset_links_tsv='dataset-links.tsv', num_threads=10):
    """Download the dataset images to out_dir, based on the links in dataset_links_tsv.

    dataset_links_tsv is expected to be a tab-separated file with two columns: image url, and local path with format
    <genre>/<album_id>.jpg
    """
#     if os.path.exists(out_dir):
#         raise ValueError('%s already exists' % out_dir)

    session = requests.Session()
    with open(dataset_links_tsv) as fh:
        jobs = []
        genres = set()
        for url, rel_path in csv.reader(fh, delimiter='\t'):#43 on Trending
Hoe Thuisbezorgd een flinke hap uit jouw pizza 
            genres.add(os.path.dirname(rel_path))
            path = os.path.join(out_dir, rel_path)
            if not os.path.exists(path):
                jobs.append((session, url, os.path.join(out_dir, rel_path)))

    for genre in genres:
        genre_path = os.path.join(out_dir, genre)
        if not os.path.exists(genre_path):
            os.makedirs(genre_path)
            
    print('Still needs to download {} covers'.format(len(jobs)))
    pool = ThreadPool(num_threads)
    num_successes = sum(pool.map(_download_image, jobs))
    print('Successfully downloaded %s/%s images' % (num_successes, len(jobs)))
    
download_dataset_images('./img/')

Still needs to download 10000 covers
Successfully downloaded 9753/10000 images


### Set labels

In [16]:
labels = [(f,i) for i, f in enumerate(os.listdir('img'))]
labels

[('hiphop_rap', 0),
 ('dubstep', 1),
 ('rock', 2),
 ('ambient', 3),
 ('pop', 4),
 ('punk', 5),
 ('jazz', 6),
 ('folk', 7),
 ('soul', 8),
 ('metal', 9)]

## Look through the downloaded Albums

In [69]:
class Album:
    
    def __init__(self, path, infos):
        self.path = path
        self.name = infos[0]
        self.genre = infos[1]
        self.label = infos[2]
        self.year = infos[3]

def read_files(file_list):
    albums = []
    fucked = []
    for i in range(len(file_list)):
        file_name = file_list[i]
        split = file_name.split('-')
        if file_name.count('-') >= 4:
            print('FUCK: ', file_name)
            fucked.append(file_name)
        else:
            albums.append(Album(os.path.abspath(file_name), split))
    return albums, fucked
    

def explore_files():
    current_folder_filename_list = os.listdir("./data") # Change this PATH to traverse other directories if you want.
    albums, fucked = [], []
    if current_folder_filename_list != None:
        print("%s files were found under current folder. " % len(current_folder_filename_list))
        albums, fucked = read_files(current_folder_filename_list)
    return albums, fucked

albums, fucked = explore_files()
genres = [a.genre for a in albums]

unique_genres = sorted(set(genres))
print("We start with {} genres which are definitely too many labels for a CNN at this point.".format(len(unique_genres)))

4064 files were found under current folder. 
FUCK:  TM404-Dub Techno-Kontra-Musik-2013.jpg
FUCK:  Knee-Deep In The North Sea-Jazz-Real World-2007.jpg
FUCK:  ZGMF-X19A-UK Bass-Infinite Machine-2015.jpg
FUCK:  Singles 93-03-Big Beat-Virgin-2003.jpg
FUCK:  Everywhere at the end of time - Stage 4-Dark Ambient-History Always Favours The Winners-2018.jpg
FUCK:  Acquiescence-Downtempo-Non-Projects-2012.jpg
FUCK:  32 Zel  Planet Shrooms-Hip-Hop--2015.jpg
FUCK:  Jenny Death-Hip-Hop-Harvest Records-2015.jpg
FUCK:  Re-Animations EP-Footwork-Astrophonica-2013.jpg
FUCK:  Czarface meets Metal Face-Hip-Hop-Silver Age-2018.jpg
FUCK:  And. IV [Inertia]-Glitch-Raster-Noton-2012.jpg
FUCK:  EP1 Drum Talking-Dub-One-Handed Music-2011.jpg
FUCK:  Alas Rattoisaa Virtaa-Neo-Psychedelia-Dekorder-2014.jpg
FUCK:  Old-Hip-Hop-Fool's Gold Records-2013.jpg
FUCK:  Inner Treaty-Neo-Psychedelia-Sun Ark Records-2012.jpg
FUCK:  volume 1 flick your tongue against your teeth and describe the present.-Hip-Hop--2017.png
FUCK

# Caffe2 Analysis

### Set variables

In [26]:
from caffe2.python import models
models = os.path.dirname(models.__file__)
MODEL = 'bvlc_googlenet', 'init_net.pb', 'predict_net.pb', 'ilsvrc_2012_mean.npy', 227
IMAGE_LOCATION = "img/ambient/a4072966.jpg"
codes =  "https://gist.githubusercontent.com/aaronmarkham/cd3a6b6ac071eca6f7b4a6e40e6038aa/raw/9edb4038a37da6b5a44c3b5bc52e448ff09bfe5b/alexnet_codes"

MEAN_FILE = os.path.join(models, MODEL[0], MODEL[3])
if not os.path.exists(MEAN_FILE):
    print("No mean file found!")
    mean = 128
else:
    print ("Mean file found!")
    mean = np.load(MEAN_FILE).mean(1).mean(1)
    mean = mean[:, np.newaxis, np.newaxis]
print("mean was set to: ", mean)

INIT_NET = os.path.join(models, MODEL[0], MODEL[1])
PREDICT_NET = os.path.join(models, MODEL[0], MODEL[2])
print('Init net file: ', INIT_NET)
print('Predict net file: ', PREDICT_NET)

with open(INIT_NET, "rb") as f:
    init_net = f.read()
with open(PREDICT_NET, "rb") as f:
    predict_net = f.read()
    
p = workspace.Predictor(init_net, predict_net)

No mean file found!
mean was set to:  128
Init net file:  /usr/lib/python3.7/site-packages/caffe2/python/models/bvlc_googlenet/init_net.pb
Predict net file:  /usr/lib/python3.7/site-packages/caffe2/python/models/bvlc_googlenet/predict_net.pb


# Clarifai Analysis

In [9]:
from clarifai.rest import ClarifaiApp
app = ClarifaiApp()

m = app.models.get('aaa03c23b3724a16a56b629203edc62c')

# app.tag_images('http://assets.rollingstone.com/assets/images/list/0e099b2214b1673fc76c6c60257b88aefe571def.jpg')

In [30]:
def get_albums_img(main_folder):
    img_folder_list = os.listdir(main_folder) # Change this PATH to traverse other directories if you want.
    albums = {}
    for folder in img_folder_list:
        print(folder)
        if folder not in albums.keys():
            albums[folder] = []
        # Retrieve images for this album
        images = os.listdir(main_folder + '/' +folder)
        for image in images:
            if image not in albums[folder]:
                albums[folder].append(image)
                
    return albums

albums = get_albums_img("./img")

def get_absolute_image_path(genre, image):
    return os.path.abspath("./img/{}/{}".format(genre, image))

hiphop_rap
dubstep
rock
ambient
pop
punk
jazz
folk
soul
metal


In [32]:
tryout = albums['ambient'][0:1]
for img in tryout:
    path = get_absolute_image_path('ambient', img)
    tags = m.predict('http://assets.rollingstone.com/assets/images/list/0e099b2214b1673fc76c6c60257b88aefe571def.jpg')

/home/krulvis/PycharmProjects/AlbumAnalyser/img/ambient/a1604573766.jpg
